In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [4]:
#connect all of the above using PIPES

chain_wo_parser  = prompt | model 
chain_w_parser = prompt | model | output_parser

In [5]:
# call the standard "invoke" function

# print(chain_wo_parser.invoke({"topic": "bears"}))
print(chain_w_parser.invoke({"topic": "bears"}))

#NOTE: with the StringOutputParser in the pipeline, we don't have to parse inside the returned object to get the content.  It just returns the content as a string.

content="Why don't bears wear shoes? \n\nBecause they have bear feet!"


KeyboardInterrupt: 

### More complex chain

And runnable mapp to supply user-provided inputs to the prompt

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey", "four score and seven years ago", "The New York Jets are my favorite team", "chris used to work at informatica"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [8]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='chris used to work at informatica'),
 Document(page_content='four score and seven years ago'),
 Document(page_content='bears like to eat honey')]

In [9]:
retriever.get_relevant_documents("Who is your favorite baseball team?")

[Document(page_content='The New York Jets are my favorite team'),
 Document(page_content='four score and seven years ago'),
 Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain.schema.runnable import RunnableMap

In [12]:
#NOTE: chain includes the RunnableMap, the propmpt, the model and the output_parser
input = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})
chain = input | prompt | model | output_parser

In [13]:
# this chain is a "Runnable" so we can use the "invoke"
chain.invoke({"question": "Where did chris work?"})

'Chris worked at Informatica.'

In [14]:
# this input is also a "Runnable" so we can use the "invoke"
#I believe everything in the chain is "runnable"
input.invoke({"question": "Where did chris work?"})

{'context': [Document(page_content='chris used to work at informatica'),
  Document(page_content='harrison worked at kensho'),
  Document(page_content='four score and seven years ago'),
  Document(page_content='bears like to eat honey')],
 'question': 'Where did chris work?'}

In [15]:
# See: Prompt is also Runnable
prompt.invoke({'context': ['foo'],
 'question': 'Where did chris work?'})

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n['foo']\n\nQuestion: Where did chris work?\n")])

### Bind

and OpenAI functions

In [16]:
#Define functions for LangChain as we did in Lesson 1
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [17]:
# in lesson 1, we passed the functions list when we created and called the Chat
# we will just instantiate the Chat as model here,
#then call it with "invoke" as part of the chain.
#that doesn't give us an opportunity to pass Functions to invoke.  
# that's what "bind" is for.
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)
runnable = prompt | model
runnable.invoke({"input": "what is the weather like in Boston?"}) #this question should trigger the function

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "BOS"\n}'}})

In [18]:
# you can override the current binding, by just calling Bind again
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [24]:
model = model.bind(functions=functions)
runnable = prompt | model #NOTE: not only do you need to re-bind, but you need to re-pipe the chain
runnable.invoke({"input": "how did the new york jets do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "New York Jets"\n}'}})

### Fallbacks

In [1]:
from langchain.llms import OpenAI
import json

In [2]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001" #Using an old language model that won't make valid JSON, which will break the chain.
)
simple_chain = simple_model | json.loads

In [3]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [4]:
simple_model.invoke(challenge)

'\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land"]\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["Ode to a Nightingale","John Keats","Thou still unravish\'d bride of quietness, Thou foster-child of silence and slow time"]'

In [ ]:
# this invoke call of the chain will FAIL.  NOTE: that the response from the model invoke is not valid JSON.
simple_chain.invoke(challenge)

In [9]:
# this one passes as it uses a more modern LLM
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [10]:
# FALLBACKS: you start with the chain you want, then create a second complete chain and identify it as the fallback
# That whole this is a runnable that can be invoked.

final_chain = simple_chain.with_fallbacks([chain])
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

### Interface

In [3]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

#### Invoke

In [4]:
# Syncronous call on ONE input

chain.invoke({"topic": "bears"})

#### Batch

In [13]:
# Syncronous call on a LIST of inputs
# These are executed in parallel to the extent possible

chain.batch([{"topic": "bears"},{"topic": "fongs"}])

#### Stream

In [ ]:
#Streams the inference of the repsonse
for t in chain.stream({"topic": "bears"}):
    print(t)

#### Async

response = await chain.ainvoke({"topic": "bears"})
response